In [2]:
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp

In [1]:
import mediapipe as mp

# Test Hands solution
mp_hands = mp.solutions.hands
print("Mediapipe imported successfully!")


Mediapipe imported successfully!


In [15]:
import cv2
import mediapipe as mp
import os
import numpy as np
import time

# Initialize MediaPipe for hand landmarks
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Create a folder for storing the data
data_folder = "sign_language_data1"
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

# Initialize OpenCV video capture
cap = cv2.VideoCapture(0)  # 0 is the default camera index

# Set up hand detection
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    label = input("Enter the label for the gesture (e.g., A, B, C): ")
    print("Press 'q' to quit.")

    frame_counter = 0
    start_time = time.time()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Convert the BGR image to RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        # Check if hand landmarks are found
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Extract landmarks as a flattened array
                landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark]).flatten()

                # Save data at 2-second intervals
                if time.time() - start_time >= 2:  # Save every 2 seconds
                    data_filename = os.path.join(data_folder, f"{label}_{frame_counter}.npy")
                    np.save(data_filename, landmarks)
                    print(f"Saved frame {frame_counter} for label {label}")
                    frame_counter += 1
                    start_time = time.time()

        # Display the frame with landmarks
        cv2.imshow("Hand Landmarks", frame)

        # Break on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()

print("Data collection completed.")


Press 'q' to quit.
Saved frame 0 for label want
Saved frame 1 for label want
Saved frame 2 for label want
Saved frame 3 for label want
Saved frame 4 for label want
Saved frame 5 for label want
Saved frame 6 for label want
Saved frame 7 for label want
Saved frame 8 for label want
Saved frame 9 for label want
Saved frame 10 for label want
Saved frame 11 for label want
Saved frame 12 for label want
Saved frame 13 for label want
Saved frame 14 for label want
Saved frame 15 for label want
Saved frame 16 for label want
Saved frame 17 for label want
Saved frame 18 for label want
Saved frame 19 for label want
Saved frame 20 for label want
Saved frame 21 for label want
Saved frame 22 for label want
Saved frame 23 for label want
Saved frame 24 for label want
Saved frame 25 for label want
Saved frame 26 for label want
Saved frame 27 for label want
Saved frame 28 for label want
Saved frame 29 for label want
Saved frame 30 for label want
Saved frame 31 for label want
Saved frame 32 for label want
S

Counter({'dad': 21, 'food': 21, 'good luck': 21, 'help': 21, 'mom': 21, 'thank you': 21, 'hello': 19, 'love you': 15, 'no': 14, 'sorry': 14, 'uncle': 14, 'home': 11, 'yes': 11})


In [7]:
import numpy as np
import os

data_folder = "sign_language_data1"
label = "sorry"  # Example label
sample_file = os.path.join(data_folder, f"{label}_0.npy")

landmarks = np.load(sample_file)
print(landmarks.shape)  # Check the shape of the data


(63,)


## Splitting the data

In [8]:
from sklearn.model_selection import train_test_split

data = []  # List to store landmarks
labels = []  # List to store corresponding labels

for file in os.listdir(data_folder):
    if file.endswith(".npy"):
        label = file.split("_")[0]  # Extract label from the filename
        landmarks = np.load(os.path.join(data_folder, file))
        data.append(landmarks)
        labels.append(label)

# Convert to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Split data
X_train, X_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [9]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_val_encoded = encoder.transform(y_val)
y_test_encoded = encoder.transform(y_test)


In [10]:
X_train = X_train.reshape((-1, 21, 3, 1))
X_val = X_val.reshape((-1, 21, 3, 1))
X_test = X_test.reshape((-1, 21, 3, 1))


In [11]:
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)


X_train shape: (430, 21, 3, 1)
X_val shape: (92, 21, 3, 1)
X_test shape: (93, 21, 3, 1)


In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(21, 3, 1)),
    MaxPooling2D(pool_size=(2, 1), padding='same'),  # Adjust pool size to match input shape

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 1), padding='same'),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(y_train_encoded)), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())


b:\mini_project\mini_project\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 21, 3, 32)      │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 11, 3, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 11, 3, 64)      │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 168,335 (657.56 KB)

 Trainable params: 168,335 (657.56 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
history = model.fit(
    X_train, y_train_encoded,
    validation_data=(X_val, y_val_encoded),
    epochs=100,
    batch_size=32,
)


In [10]:
# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded)
print("Test Accuracy:", test_accuracy)



3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9365 - loss: 0.1895
Test Accuracy: 0.9354838728904724


### load and save the model

In [ ]:
# Save the model after training
model.save("gesture_recognition_model.h5")

# Load the model
from tensorflow.keras.models import load_model
model = load_model("gesture_recognition_model.h5")


In [13]:
import os

# Directory containing your training data
data_folder = "sign_language_data1"

# Extract labels dynamically
labels = sorted({filename.split('_')[0] for filename in os.listdir(data_folder) if filename.endswith('.npy')})
label_map = {i: label for i, label in enumerate(labels)}

print("Generated Label Map:", label_map)
# Example output: {0: 'A', 1: 'B', 2: 'C'}

Generated Label Map: {0: 'dad', 1: 'good morning', 2: 'hello', 3: 'help', 4: 'i', 5: 'love you', 6: 'me', 7: 'mom', 8: 'need', 9: 'no', 10: 'pineapple', 11: 'sorry', 12: 'want', 13: 'yes', 14: 'your'}


In [10]:
from joblib import dump, load

# Save the model
dump(model, "model.joblib")

# Load the model
loaded_model = load("model.joblib")


b:\mini_project\mini_project\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [12]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

# Hand detection setup
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Convert the frame to RGB for MediaPipe
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect hand landmarks
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                        landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark]).flatten()
                        landmarks = np.reshape(landmarks, (1, 21, 3, 1))

        # Predict the gesture
                        prediction = model.predict(landmarks)
                        predicted_label_index = np.argmax(prediction)

        # Map numeric label to gesture
                        predicted_sign = label_map.get(predicted_label_index, "Unknown")

        # Display the prediction
                        cv2.putText(frame, f"Sign: {predicted_sign}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)



        # Show the frame
        cv2.imshow("Sign Language Detection", frame)

        # Break on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━

In [11]:
import pickle

# Save the label_map to a file
with open("label_map.pkl", "wb") as f:
    pickle.dump(label_map, f)

# Later, load the label_map during testing
with open("label_map.pkl", "rb") as f:
    label_map = pickle.load(f)
